```{contents}
```
## Unit Testing

Unit testing in LangGraph focuses on **verifying correctness, determinism, safety, and stability** of individual graph components—**nodes, state transitions, routing logic, tools, and subgraphs**—before they are composed into production systems.

LangGraph’s state-machine model makes it **uniquely testable**, because each node is a **pure transformation of state**.

---

### **1. What Is a “Unit” in LangGraph?**

| Test Unit        | What is being tested                  |
| ---------------- | ------------------------------------- |
| Node             | Single function / LLM / tool behavior |
| State Transition | Correct update of state               |
| Routing Logic    | Conditional edge decisions            |
| Reducer          | State merge behavior                  |
| Tool Integration | Tool input/output contract            |
| Subgraph         | Isolated workflow behavior            |

---

### **2. Testing Strategy Layers**

```
Node Tests → Routing Tests → Subgraph Tests → Full Graph Tests
```

Each layer increases integration scope.

---

### **3. Testing a Node**

#### Example Node

```python
def add_one(state):
    return {"count": state["count"] + 1}
```

#### Unit Test

```python
def test_add_one():
    state = {"count": 1}
    new_state = add_one(state)
    assert new_state["count"] == 2
```

**Principle:**
Nodes should behave like **pure functions**.

---

### **4. Testing Conditional Routing**

#### Router

```python
def router(state):
    return "end" if state["count"] >= 5 else "loop"
```

#### Test

```python
def test_router():
    assert router({"count": 3}) == "loop"
    assert router({"count": 5}) == "end"
```

---

### **5. Testing a Subgraph**

```python
graph = builder.compile()
```

```python
def test_subgraph_execution():
    result = graph.invoke({"count": 0})
    assert result["count"] == 5
```

This validates:

* loop correctness
* stopping condition
* state integrity

---

### **6. Mocking LLMs and Tools**

LLMs must **never** be called in unit tests.

```python
def fake_llm(state):
    return {"answer": "mocked"}
```

Replace real nodes:

```python
builder.add_node("llm", fake_llm)
```

This yields **deterministic tests**.

---

### **7. Testing Failure & Recovery**

```python
def faulty_node(state):
    raise ValueError("boom")
```

```python
def test_error_handling():
    try:
        graph.invoke({"count": 0})
    except ValueError:
        assert True
```

---

### **8. Testing Cycles Safely**

```python
result = graph.invoke({"count": 0}, config={"recursion_limit": 10})
```

Prevents infinite loops during testing.

---

### **9. Testing Reducers**

```python
def reducer(a, b):
    return a + b
```

```python
def test_reducer():
    assert reducer(2, 3) == 5
```

---

### **10. Continuous Integration Checklist**

| Area    | Verify              |
| ------- | ------------------- |
| Nodes   | Pure, deterministic |
| Routing | All branches tested |
| State   | Correct updates     |
| Loops   | Terminate safely    |
| Tools   | Mocked              |
| Errors  | Handled             |

---

### **11. Why This Matters**

LangGraph enables **stateful AI systems**.
Without unit tests, failures become **non-deterministic, untraceable, and dangerous** in production.

Unit testing transforms LangGraph workflows into **reliable software systems**, not experiments.


### Demonstration

In [1]:
# ====== One-Cell LangGraph Unit Testing Demo ======

from typing import TypedDict
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Define State
# ---------------------------
class State(TypedDict):
    count: int
    answer: str

# ---------------------------
# 2. Nodes
# ---------------------------
def add_one(state):
    return {"count": state["count"] + 1}

def router(state):
    return "stop" if state["count"] >= 3 else "loop"

def fake_llm(state):
    return {"answer": "mocked_output"}

# ---------------------------
# 3. Build Graph
# ---------------------------
builder = StateGraph(State)

builder.add_node("inc", add_one)
builder.add_node("llm", fake_llm)

builder.set_entry_point("inc")
builder.add_edge("inc", "llm")

builder.add_conditional_edges("llm", router, {
    "loop": "inc",
    "stop": END
})

graph = builder.compile()

# ---------------------------
# 4. Unit Tests
# ---------------------------

# Test node
assert add_one({"count": 1, "answer": ""})["count"] == 2

# Test router
assert router({"count": 2, "answer": ""}) == "loop"
assert router({"count": 3, "answer": ""}) == "stop"

# Test full graph execution
result = graph.invoke({"count": 0, "answer": ""}, config={"recursion_limit": 10})

assert result["count"] == 3
assert result["answer"] == "mocked_output"

print("All LangGraph unit tests passed successfully.")


All LangGraph unit tests passed successfully.
